In [44]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


pyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html),
all of which include datasets.

The torchvision.datasets module contains Dataset objects for many real-world vision data like
CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and
target_transform to modify the samples and labels respectively.

A PyTorch Tensor is basically the same as a numpy array: it does not know anything about deep learning or computational graphs or gradients, and is just a generic n-dimensional array to be used for arbitrary numeric computation.

The biggest difference between a numpy array and a PyTorch Tensor is that a PyTorch Tensor can run on either CPU or GPU. To run operations on the GPU, just cast the Tensor to a cuda datatype.

In [45]:
training_data = datasets.FashionMNIST(root='data',train=True, download=True,transform=ToTensor())

In [46]:
test_data = datasets.FashionMNIST(root='data',train=False, download=True, transform= ToTensor())

PyTorch has two [primitives to work with data](https://pytorch.org/docs/stable/data.html):
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.

In [47]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

![](https://docs.nvidia.com/deeplearning/performance/dl-performance-convolutional/graphics/convo-tensor.svg)

In [48]:
for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [49]:
device = 'cuda' if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cpu


In [50]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [51]:
model = NeuralNetwork()

In [52]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [53]:
def train(dataloader, model, loss_fn, optimizer):
  
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):

    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0 :
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [54]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    printtt = True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [55]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309022  [    0/60000]
loss: 2.292829  [ 6400/60000]
loss: 2.271144  [12800/60000]
loss: 2.264394  [19200/60000]
loss: 2.248418  [25600/60000]
loss: 2.212064  [32000/60000]
loss: 2.232138  [38400/60000]
loss: 2.193746  [44800/60000]
loss: 2.180630  [51200/60000]
loss: 2.159911  [57600/60000]
Test Error: 
 Accuracy: 45.5%, Avg loss: 2.149940 

Epoch 2
-------------------------------
loss: 2.160389  [    0/60000]
loss: 2.149143  [ 6400/60000]
loss: 2.087502  [12800/60000]
loss: 2.112666  [19200/60000]
loss: 2.059658  [25600/60000]
loss: 1.993497  [32000/60000]
loss: 2.039924  [38400/60000]
loss: 1.953670  [44800/60000]
loss: 1.947925  [51200/60000]
loss: 1.897506  [57600/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.881669 

Epoch 3
-------------------------------
loss: 1.913232  [    0/60000]
loss: 1.882352  [ 6400/60000]
loss: 1.757978  [12800/60000]
loss: 1.817839  [19200/60000]
loss: 1.702829  [25600/60000]
loss: 1.644293  [32000/600

In [56]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [57]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [58]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(pred[0])
    print(pred[0].argmax())
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([-1.7399, -2.3298, -0.9606, -1.7353, -1.1970,  2.2471, -1.1413,  2.5336,
         1.6606,  2.9165])
tensor(9)
Predicted: "Ankle boot", Actual: "Ankle boot"
